In [2]:
import numpy as np 
import pandas as pd 
import scipy.integrate

In [3]:

def load_constants():
    """Returns constants frequently used in this work"""
    params = {'vtl_max': 20,  # Max translation speed in AA/s
              'm_Rb': 7459,  # Proteinaceous mass of ribosome  in AA
              'Kd_cpc': 0.03,  # precursor dissociation constant in abundance units
              'Km': 5E-4,  # Nutrient monod constant in M
              'Y': 2.95E19,  # Yield coefficient
              'OD_conv': 1.5E17,  # Conversion factor from OD to AA mass.
              'Km_u': 3E-5,  # uncharged tRNA dissociation constant in abundance units
              'Kd_c': 3E-5,  # Charged tRNA dissociation constant in abundance units
              # Maximum tRNA synthesis rate  in abundance units per unit time
              'kappa_max': (64 * 5 * 3600) / 1E9,
              'tau': 1,  # ppGpp threshold parameter for charged/uncharged tRNA balance
              # Fraction of proteome deoveted to `other` proteins for E. coli.
              'phi_O': 0.55,
              }
    params['gamma_max'] = params['vtl_max'] * 3600 / params['m_Rb']
    return params

In [5]:
def unpack_params(params, args): 
    n_nuts = len(args['n_nutrients'])
    n_species = len(args(['n_species']))
    n_params = n_nuts + 4 # 4 is hard coded to account for M_O, M_Rb, tRNA_u, tRNA_c

    # Unpack the nutrient concentrations
    nutrients = [c for c in params[-n_nuts:]]
    masses = params[:-n_nuts]

    # Separate the params into species
    species = []
    for i in range(n_species):
        species.append(masses[i*n_params:n_params * (i + 1)])

    return nutrients, species

def compute_FPM(tRNA_u, tRNA_c, c_nt, args):
    n_nuts = len(c_nt)
    # Set ribosomal allocation, based on the ratio and enforcing positivity
    if tRNA_u <= 0:
        phi_Rb = (1 - args['phi_O'])
    else:
        ratio = tRNA_c/tRNA_u
        phi_Rb = (1 - args['phi_O']) * (ratio / (ratio + args['tau']))

    # Set the hierarchy and metabolic allocation 
    metabolic_suballocation = [] 
    if args['alpha_mode'] == 'dynamic':
        # We assume a monod function for the suballocation
        for i in range(n_nuts - 1):
            monod_fn = c_nt[i] / (c_nt[i] + args['alpha_Km'][i])
            if i == 0:
                metabolic_suballocation[i] = monod_fn
            else:
                metabolic_suballocation[i] = (1 - np.sum(metabolic_suballocation[:i]) * monod_fn)
        metabolic_suballocation[-1] = 1 - np.sum(metabolic_suballocation)

    elif args['alpha_mode'] == 'fixed':
        metabolic_suballocation = args['alpha']

    # Compute the rates
    gamma = args['gamma_max'] * tRNA_c / (tRNA_c + args['Km_c'])
    kappa = args['kappa_max'] * phi_Rb / (1 - args['phi_O'])
    nu = np.zeros_like(c_nt) 
    for i, c in enumerate(c_nt):
        nu[i] = args['nu_max'][i] * (tRNA_u / (tRNA_u + args['Km_u'])) * (c_nt[i] / (c_nt[i] + args['Km'][i])) 
 
    # Compute the net metabolic allocation
    phi_Mb = (1 - args['phi_O'] - phi_Rb) * metabolic_suballocation
    return phi_Rb, phi_Mb, nu, gamma, kappa

   
def compute_derivatives(masses, c_nt, args):
    """Computes the flux-parity system for a single community member""" 
    n_nuts = len(c_nt)
    M = np.sum(masses[:-2])

    # Unpack the metabolic masses
    M_Mb = masses[:n_nuts]

    # Unpack the ribosomal masse and charged, uncharged tRNA concentrations
    M_Rb, _,  tRNA_u, tRNA_c = masses[n_nuts:]   

    # Compute the flux-parity properties and unpack
    phi_Rb, phi_Mb, nu, gamma, kappa = compute_FPM(tRNA_u, tRNA_c, c_nt, args)

    # Compute the fluxes
    J_Rb = M_Rb * gamma / M
    J_Mb = np.sum(nu * M_Mb) / M

    # Compute the derivatives
    dM_Mb = phi_Mb * J_Rb
    dM_Rb = phi_Rb * J_Rb
    dM_O = args['phi_O'] * J_Rb
    dtRNA_u = kappa + J_Rb * (1 - tRNA_u) - J_Mb
    dtRNA_c = J_Mb - J_Rb * (1 + tRNA_c)
    dc_nt = -J_Mb * M/args['Y']
    return [dM_Mb, dM_Rb, dM_O, dtRNA_u, dtRNA_c], dc_nt


def ecosystem_dynamics(t, params, args):
    """Computes the dynamics of a community of self replicators"""
    ecosystem_args = args['ecosystem']
    community_args = args['community']
    nutrients, species = unpack_params(params, ecosystem_args)
    mass_derivatives = []
    nutrient_derivatives = np.zeros(len(nutrients))
    for i, s in enumerate(species):
        dM, dc = compute_derivatives(s, nutrients, community_args[i])
        for m in dM:
            mass_derivatives.append(m)
        for i, c in enumerate(dc):
            nutrient_derivatives[i] += c
    du = np.array([np.array(mass_derivatives), nutrient_derivatives]).flatten()     
    return du


class Ecosystem():
    def __init__(self, n_species, n_nutrients):
        self.n_species = n_species
        self.n_nutrients = n_nutrients



In [6]:
Ecosystem(4, 2)

In [83]:
class FluxParityAllocator():
    """Base class for a self replicator obeying flux-parity allocation."""
    def __init__(self, suballocation, constants={}):
        """
        Instantiates a self replicating organism that undergoes flux-parity regulation 
        of its resource allocation. 

        Parameters
        ----------
        suballocation : dict 
            A dictionary defining the metabolic suballocation strategy of the 
            self replicator. Must have the following keys:
                strategy: str, `'dynamic'` or `'static'`
                    The type of suballocation. If `'dynamic'`, the suballocation
                    of each metabolic sector follows a Monod function with a 
                    Monod constant `K` and sensitivty `n`. If `'static'`,
                    suballocation is deemed to be at a fixed value, supplied
                    with key `alpha.`    
                metabolic_rates: numpy.ndarray of floats [0, inf)
                    The metabolic rate for consumption of each nutrient in units 
                    of inverse hours.
                alpha : `numpy.ndarray` of float, [0, 1]
                    The fixed suballocation of the metabolic strategy. Values 
                    must sum to `1.0`. This is only used if `strategy: 'static'` 
                    is supplied.
                K : numpy.ndarray of float [0, inf)
                    The Monod constant for the suballocation, in units of concentration.
                    This is only used if `strategy: 'dynamic'` is supplied.
                n : numpy.ndarray of int [1, inf) 
                    The sensitivity parameter of the Monod function. This is only 
                    used if `strategy: 'dynamic'` is supplied.

               
        constants : dict, optional
            A dictionary of the constant model parameters. Default values
            (described below) can be overridden by providing a key,value pair.
                gamma_max : float
                    The maximum translation rate in inverse hours. Default 
                    value is 9.65 inv. hr. 
                phi_O : float [0, 1]
                    The allocation towards all non-ribosomal and non-metabolic 
                    proteins. Default value is 0.55. 
               tau : float [0, inf)                               
                    The sensitivity parameter of the flux-parity ribosomal 
                    allocation function. Default is 1.0
               kappa_max : float [0, inf)
                    The maximum transcription rate of uncharged tRNA in 
                    mass abundance units per hour. Default is 1.15E-3 inverse hr.
               Km_u : float [0, inf)                
                    The Michaelis-Menten constant for binding of uncharged tRNA 
                    to the metabolic components in units of relative mass abundance. 
                    Default value is 3E-5.
               Km_c : float [0, inf)                
                    The Michaelis-Menten constant for binding of charged tRNA 
                    to the ribosomal components in units of relative mass abundance. 
                    Default value is 3E-5.
               Km : numpy.ndarray [0, inf)
                    The Monod constant for metabolic activity as a function of
                    the external nutrient concentration in units of concentration. 
                    Default is the same for each nutrient with 5E-6 M.
               Y : numpy.ndarray [0, inf)
                    The yield coefficient for growth on each nutrient. Default 
                    is 2.95E19 for each nutrient supplied.              

        """
        self.num_metab = len(suballocation['nu_max'])

        # Set the properties of the self replicator.
        _constants = {'gamma_max': 9.65,
                     'phi_O': 0.55,
                     'tau': 1,
                     'kappa_max':1.15E-3,
                     'Km_u': 3E-5,
                     'Km_c': 3E-5,
                     'Km': [5E-6 for _ in range(self.num_metab)],
                     'Y': [2.95E19 for _ in range(self.num_metab)],
                     'nu_max': suballocation['nu_max']}
        self._overridden_pars = {}
        for k, v in constants.items():
            _constants[k] = v
            self._overridden_pars[k] = v

        # Set attributes from the constant dictionary.
        for k, v in _constants.items():
            setattr(self, k, v)

        # Set the suballocation details
        self.strategy = suballocation['strategy']
        if self.strategy == 'static':
            if np.sum(suballocation['alpha']) != 1:
                raise ValueError(f"Suballocation parameters must sum to 1!")
            self.alpha = suballocation['alpha'] 
        elif self.strategy == 'dynamic':
            self.K = suballocation['K']
            self.n = suballocation['n']
        else:
            raise ValueError("Supplied metabolic strategy must be either `static` or `dynamic`.") 

    def __repr__(self):
        rep = f"""
================Self Replicator================
metabolic_strategy          : {self.strategy}
number of metabolic classes : {self.num_metab}
metabolic rates             : {self.nu_max}
"""
        if self.strategy == 'dynamic':
            rep += f"\tK's [M] : {self.K}"
            rep += f"\n\tn's : {self.n}"
        elif self.strategy == 'static':
            rep += f"\talpha's: {self.alpha}"
        if len(self._overridden_pars) > 0:
            rep += f"\nOverridden Default Parameters:" 
            for k, v in self._overridden_pars.items():
                rep += f"\n\t{k}: {v}"
        if self._properties:
            rep+=f"""\n
----------------Allocation------------------
tRNA ratio (charged/uncharged) : {self.ratio}
gamma (translation rate)       : {self.gamma} [inv. hr.]
nu (metabolic rate)            : {self.nu} [inv. hr.]
kappa (transcription rate)     : {self.kappa} [inv. hr.]
phi_Rb (ribosomal allocation)  : {self.phi_Rb}   
phi_Mb (metabolic allocation)  : {self.phi_Mb} 

            """
        return rep 

    def compute_properties(self, tRNA_c, tRNA_u, nutrients):
        """
        Computes the self replicator properties, including the allocation parameters 
        and the corresponding rates. 

        # Parameters
        # ----------
        # tRNA_c : float [0, inf)
        #     The charged tRNA concentration in relative mass abundance units.
        # tRNA_u : float[0, inf) 
        #     The uncharged tRNA concentration in relative mass abundance units.
        # nutrients : numpy.ndarray float 
        #     The concentration of the nutrients in the environment for calculation
        #     of rates. 
        """

        # Determine if the uncharged tRNA concentration is 0. If so, set the 
        # ribosomal allocation to 1 and do not compute the ratio
        if tRNA_u <= 0: #Include less than zero for numerical underflow
            self.phi_Rb = (1 - self.phi_O)
            self.ratio = np.inf
        else:
            self.ratio = tRNA_c / tRNA_u
            self.phi_Rb = (1 - self.phi_O) * (self.ratio / (self.ratio + self.tau))

        # Adjust the flux parity regulation on transcription
        self.kappa = self.kappa_max * self.phi_Rb / (1 - self.phi_O)

        # Set the translation rate
        self.gamma = self.gamma_max * tRNA_c / (tRNA_c + self.Km_c)
        
        # Set the metabolic rates
        self.nu  = np.zeros_like(self.nu_max)
        metab_factor = tRNA_u / (tRNA_u + self.Km_u)
        for i in range(self.num_metab):
            env_factor = nutrients[i] / (nutrients[i] + self.Km[i])
            self.nu[i] = self.nu_max[i] * env_factor * metab_factor

        # Compute the metabolic allocation
        self.phi_Mb = np.zeros(self.num_metab)
        if self.strategy == 'static':
            # Set the fixed suballocations as prescribed by the alpha parameters.
            for i in range(self.num_metab):
                self.phi_Mb[i] = (1 - self.phi_O - self.phi_Rb) * self.alpha[i] 

        elif self.strategy == 'dynamic':
            # set the suballocation based on the nutrient concentrations
            occupied_phi_Mb = 0 
            for i in range(self.num_metab):
                if i == (self.num_metab - 1):
                    # Evaluate the remaining suballocation to the final nutrient in the hierarchy.
                    self.phi_Mb[i] = (1 - self.phi_O - self.phi_Rb - occupied_phi_Mb) 
                else:
                    # Set the suballocation given the supplied monod function properties
                    numer = (nutrients[i] / self.K[i])**self.n
                    factor = numer / (numer + 1)
                    self.phi_Mb[i] =  (1 - self.phi_O - self.phi_Rb - occupied_phi_Mb) * factor
                    occupied_phi_Mb += self.phi_Mb
        self._properties = True

    def compute_derivatives(self, masses, nutrients):
        """
        Computes the mass and concentration derivatives given the internal flux-parity 
        allocation state.

        Parameters
        ----------
        masses : numpy.ndarray 
            The masses of the protein sectors and the concentrations of the charged 
            and uncharged tRNAs.
             
        """


In [ ]:
class SelfReplicator():
    """Base class for self replication"""
    def __init__(self, variables, allocation):
        """
        Initializes a self-replicating system obeying flux-parity regulation for 
        """

In [84]:
cell = FluxParityAllocator({'strategy': 'static', 'alpha':[0.75, 0.25, 0], 'nu_max':[10, 5, 2]})

In [85]:
cell.compute_properties(1E-5, 1E-5, [0.001, 0.001, 0.001])

0.16874999999999998
0.056249999999999994
0.0


In [86]:
cell

# masses = np.array([0, 0, 1, 1, 1E-5, 1E-5])
# nutrients = np.array([1E-5, 1E-5, 1E-5])
# M_Rb, M_Mb, tRNA_u, tRNA_c = compute_derivatives(masses, nutrients, {})
# phi_Rb, phi_Mb = allocation(tRNA_u, tRNA_c, nutrients, {'phi_O':0.55, 'tau':1, 'alpha_mode':'fixed', 'alpha':np.array([0.6, 0.2, 0.2])})


================Self Replicator================
metabolic_strategy          : static
number of metabolic classes : 3
metabolic rates             : [10, 5, 2]
	alpha's: [0.75, 0.25, 0]

----------------Allocation------------------
tRNA ratio (charged/uncharged) : 1.0
gamma (translation rate)       : 2.4125 [inv. hr.]
nu (metabolic rate)            : [2 1 0] [inv. hr.]
kappa (transcription rate)     : 0.000575 [inv. hr.]
phi_Rb (ribosomal allocation)  : 0.22499999999999998   
phi_Mb (metabolic allocation)  : [0.16875 0.05625 0.     ] 

            

In [49]:
params


def load_constants():
    """Returns constants frequently used in this work"""
    params = {'vtl_max': 20,  # Max translation speed in AA/s
              'm_Rb': 7459,  # Proteinaceous mass of ribosome  in AA
              'Kd_cpc': 0.03,  # precursor dissociation constant in abundance units
              'Km': 5E-4,  # Nutrient monod constant in M
              'Y': 2.95E19,  # Yield coefficient
              'OD_conv': 1.5E17,  # Conversion factor from OD to AA mass.
              'Km_u': 3E-5,  # uncharged tRNA dissociation constant in abundance units
              'Kd_c': 3E-5,  # Charged tRNA dissociation constant in abundance units
              # Maximum tRNA synthesis rate  in abundance units per unit time
              'kappa_max': (64 * 5 * 3600) / 1E9,
              'tau': 1,  # ppGpp threshold parameter for charged/uncharged tRNA balance
              # Fraction of proteome deoveted to `other` proteins for E. coli.
              'phi_O': 0.55,
              }
    params['gamma_max'] = params['vtl_max'] * 3600 / params['m_Rb']

NameError: name 'params' is not defined

In [28]:
phi_Mb

array([0.135, 0.045, 0.045])

In [15]:
tRNA_c

1e-05

In [16]:
M_Mb

[0, 0, 1]